In [1]:
%pip install itables tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [167]:
import math
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path
from tqdm import tqdm
from scipy.stats import entropy
from itertools import permutations


def safe_log2(x):
    return math.log2(x) if x > 0 else 0


def tuple_to_string(num_tuple):
    return ''.join(map(str, num_tuple))


def calculate_bulls_cows(source, target):
    if len(source) != len(target):
        raise ValueError("Input arrays must have the same length")

    bulls = sum(s == t for s, t in zip(source, target))
    common_digits = set(source) & set(target)
    cows = sum(min(source.count(digit), target.count(digit)) for digit in common_digits) - bulls

    return bulls, cows


def parse_bulls_n_cows_map_name(digits, guesses={}):
	suffix = ''.join(f'_{k}:{v[0]}{v[1]}' for (k, v) in sorted(guesses.items()))
	return f'bulls_n_cows_map/{digits}{suffix}.pkl'


def initialize(originals, digits):
    filepath = parse_bulls_n_cows_map_name(digits=digits)
    if os.path.isfile(filepath):
        return

    bulls_n_cows_map = {}
    for i, _ in enumerate(originals):
        bulls_n_cows_map[i] = {}
        for di in range(digits+1):
            for dj in range(di+1):
                bulls_n_cows_map[i][(dj, di-dj)] = set()

    for i, source in tqdm(enumerate(originals), total=len(originals)):
        bulls_n_cows_map[i][(digits, 0)].add(i)
        for j in range(i):
            target = originals[j]
            bulls_n_cows = calculate_bulls_cows(source, target)
            bulls_n_cows_map[i][bulls_n_cows].add(j)
            bulls_n_cows_map[j][bulls_n_cows].add(i)

    with open(filepath, 'wb') as f:
    	pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)


def convert_bulls_n_cows_map(originals, bulls_n_cows_map):
	return {originals[i]: {bc: set(originals[j] for j in bulls_n_cows_map[i][bc]) for bc in bulls_n_cows_map[i] if len(bulls_n_cows_map[i][bc]) > 0} for i in bulls_n_cows_map}


def read_bulls_n_cows_map(digits, curr_guesses={}):
	filepath = parse_bulls_n_cows_map_name(digits, curr_guesses)
	if not os.path.isfile(filepath):
		return None

	with open(filepath, 'rb') as f:
		bulls_n_cows_map = pickle.load(f)

	return bulls_n_cows_map


def update_bulls_n_cows_map(org_idx_map, guess, bulls_n_cows, digits, curr_guesses={}):
    next_guesses = curr_guesses.copy()
    next_guesses[guess] = bulls_n_cows

    filepath = parse_bulls_n_cows_map_name(digits=digits, guesses=next_guesses)
    if os.path.isfile(filepath):
        return read_bulls_n_cows_map(digits=digits, curr_guesses=next_guesses)

    bulls_n_cows_map = read_bulls_n_cows_map(digits=digits, curr_guesses=curr_guesses)

    guess_idx = org_idx_map[guess]	
    candidates = bulls_n_cows_map[guess_idx][bulls_n_cows]

    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            bulls_n_cows_map[src_idx][bc] = bulls_n_cows_map[src_idx][bc].intersection(candidates)

    with open(filepath, 'wb') as f:
        pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)
        
    return bulls_n_cows_map


def calc_candidates(bulls_n_cows_map):
    candidates = set()
    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            candidates = candidates.union(bulls_n_cows_map[src_idx][bc])

    return candidates


def calc_entropy_score_map(bulls_n_cows_map, candidates, candidate_entropy, guess_count):
    entropy_map = {}
    score_map = {}
    
    def calc_score(entropy):
        return np.sqrt(entropy)

    C = len(candidates)
    for idx in bulls_n_cows_map:
        factor = (1 - 1/C) if idx in candidates else 1
        entropy_map[idx] = entropy([len(bulls_n_cows_map[idx][bc]) for bc in bulls_n_cows_map[idx]], base=2)
        score_map[idx] = guess_count + calc_score(candidate_entropy - entropy_map[idx]) * factor

    return entropy_map, score_map


def guess_based_on_score(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, candidate_entropy, verbose=False):
    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)

    guesses[guess] = bulls_n_cows
    candidates = calc_candidates(bulls_n_cows_map=bulls_n_cows_map)
    entropy_map, score_map = calc_entropy_score_map(bulls_n_cows_map=bulls_n_cows_map, candidates=candidates, candidate_entropy=candidate_entropy, guess_count=len(guesses))
    best_guess = min(score_map, key=score_map.get)

    print("💬", guesses)
    print("🎯", sorted([originals[c] for c in candidates]))

    if verbose:
        candidate_map = {}
        for idx in score_map:
            score = f'{score_map[idx]:.2f}P'
            if score in candidate_map:
                candidate_map[score].add(originals[idx])
            else:
                 candidate_map[score] = set([originals[idx]])

        print(f"🎲 {safe_log2(len(candidates)):.2f}B - {entropy_map[best_guess]:.2f}B | {score_map[best_guess]:.2f}P ('{originals[best_guess]}')")
        print(sorted(candidate_map.items(), key = lambda item: item[0]), end="\n\n")

    return len(candidates), originals[best_guess], entropy_map[best_guess]


def guess_based_on_entropy(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, candidate_entropy, verbose=False):
    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)

    guesses[guess] = bulls_n_cows
    candidates = calc_candidates(bulls_n_cows_map=bulls_n_cows_map)
    entropy_map, score_map = calc_entropy_score_map(bulls_n_cows_map=bulls_n_cows_map, candidates=candidates, candidate_entropy=candidate_entropy, guess_count=len(guesses))
    best_guess = max(entropy_map, key=entropy_map.get)

    print("💬", guesses)
    print("🎯", sorted([originals[c] for c in candidates]))
    
    if verbose:
        candidate_map = {}
        for idx in entropy_map:
            entropy = f'{entropy_map[idx]:.2f}B'
            if entropy in candidate_map:
                candidate_map[entropy].add(originals[idx])
            else:
                 candidate_map[entropy] = set([originals[idx]])
        
        print(f"🎲 {safe_log2(len(candidates)):.2f}B - {entropy_map[best_guess]:.2f}B | {score_map[best_guess]:.2f}P ('{originals[best_guess]}')")
        print(sorted(candidate_map.items(), key = lambda item: item[0]), end="\n\n")


    return len(candidates), originals[best_guess], entropy_map[best_guess]

In [162]:
class BullsNCows:
    def __init__(self, digits=4, guess_algorithm=guess_based_on_score, verbose=False):
        permute = permutations([i for i in range(10)], digits)
        self.originals = [tuple_to_string(p) for p in list(permute)]
        self.org_idx_map = {org: idx for idx, org in enumerate(self.originals)}
        self.digits = digits
        self.guess = guess_algorithm
        self.verbose = verbose
        self.reset()

        initialize(originals=self.originals, digits=4)


    def reset(self):
        self.guesses = {}
        self.secret = random.choice(self.originals)
        self.summary = [{
            "guess": "",
            "guess_result": "",
            "guess_actual_entropy": np.NaN,
            "candidate_count": len(self.originals),
            "candidate_entropy": safe_log2(len(self.originals)),
            "best_guess": "",
            "best_guess_entropy": np.NaN,
        }]


    def next(self):
        if len(self.guesses) > 0:
            guess = self.summary[-1]['best_guess']
        else:
            guess = random.choice(self.originals)
 
        guess_result = calculate_bulls_cows(self.secret, guess)
        
        candidate_count, best_guess, best_guess_entropy = self.guess(originals=self.originals, org_idx_map=self.org_idx_map, guesses=self.guesses, guess=guess, bulls_n_cows=guess_result, digits=self.digits, candidate_entropy=self.summary[-1]['candidate_entropy'], verbose=self.verbose)

        self.summary.append({
            "guess": guess,
            "guess_result": guess_result,
            "guess_actual_entropy": self.summary[-1]['candidate_entropy']-safe_log2(candidate_count),
            "candidate_count": candidate_count,
            "candidate_entropy": safe_log2(candidate_count),
            "best_guess": best_guess,
            "best_guess_entropy": best_guess_entropy,
        })

        return self


    def play(self, n_iter=10):
        self.reset()
        print(self.secret)
        while self.summary[-1]['guess'] != self.secret and n_iter > 0:
            self.next()
            n_iter -= 1

        return pd.DataFrame.from_dict(self.summary)

In [168]:
game = BullsNCows(digits=2)
C = len(game.originals)
C, _, _ = guess_based_on_score(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='54', bulls_n_cows=(1, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_score(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='12', bulls_n_cows=(0, 1), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_score(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='24', bulls_n_cows=(2, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)

💬 {'54': (1, 0)}
🎯 ['04', '14', '24', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64', '74', '84', '94']
🎲 4.00B - 1.54B | 3.09P ('04')
[('3.09P', {'58', '34', '56', '04', '52', '51', '14', '84', '59', '74', '94', '57', '64', '50', '24', '53'}), ('3.22P', {'15', '65', '75', '47', '48', '46', '85', '49', '42', '40', '25', '05', '95', '43', '41', '35'}), ('3.33P', {'07', '01', '20', '36', '82', '23', '91', '71', '10', '98', '70', '09', '76', '21', '12', '17', '73', '03', '27', '02', '26', '67', '90', '92', '87', '28', '69', '29', '39', '80', '62', '89', '68', '18', '32', '30', '83', '38', '06', '31', '13', '08', '72', '60', '81', '61', '16', '93', '86', '97', '79', '63', '37', '19', '96', '78'}), ('3.55P', {'54', '45'})]

💬 {'54': (1, 0), '12': (0, 1)}
🎯 ['24', '51']
🎲 1.00B - 1.00B | 2.87P ('24')
[('2.87P', {'24', '51'}), ('3.73P', {'01', '15', '56', '75', '20', '84', '59', '82', '74', '23', '95', '57', '64', '91', '71', '53', '10', '34', '17', '42', '40', '27', '02', '26', '

In [169]:
game = BullsNCows(digits=2)
C = len(game.originals)
C, _, _ = guess_based_on_entropy(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='54', bulls_n_cows=(1, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_entropy(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='12', bulls_n_cows=(0, 1), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_entropy(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='24', bulls_n_cows=(2, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)

💬 {'54': (1, 0)}
🎯 ['04', '14', '24', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64', '74', '84', '94']
🎲 4.00B - 1.54B | 3.09P ('04')
[('0.00B', {'54', '45'}), ('1.06B', {'07', '01', '20', '36', '82', '23', '91', '71', '10', '98', '70', '09', '76', '21', '12', '17', '73', '03', '27', '02', '26', '67', '90', '92', '87', '28', '69', '29', '39', '80', '62', '89', '68', '18', '32', '30', '83', '38', '06', '31', '13', '08', '72', '60', '81', '61', '16', '93', '86', '97', '79', '63', '37', '19', '96', '78'}), ('1.54B', {'15', '56', '75', '84', '59', '74', '95', '57', '64', '53', '34', '42', '40', '94', '43', '50', '41', '35', '52', '51', '47', '48', '05', '58', '49', '04', '65', '14', '46', '85', '25', '24'})]

💬 {'54': (1, 0), '12': (0, 1)}
🎯 ['24', '51']
🎲 1.00B - 1.00B | 3.73P ('01')
[('0.00B', {'07', '36', '98', '70', '09', '76', '21', '12', '73', '03', '67', '90', '54', '87', '69', '39', '80', '89', '68', '83', '30', '38', '06', '08', '60', '93', '86', '97', '79', '63', '37

In [171]:
df = BullsNCows(digits=4, guess_algorithm=guess_based_on_score, verbose=False).play(); df

6410
💬 {'8256': (0, 1)}
🎯 ['0123', '0124', '0127', '0129', '0132', '0135', '0138', '0142', '0145', '0148', '0163', '0164', '0167', '0169', '0172', '0175', '0178', '0183', '0184', '0187', '0189', '0192', '0195', '0198', '0312', '0315', '0318', '0321', '0324', '0327', '0329', '0342', '0345', '0348', '0361', '0364', '0367', '0369', '0372', '0375', '0378', '0381', '0384', '0387', '0389', '0392', '0395', '0398', '0412', '0415', '0418', '0421', '0423', '0427', '0429', '0432', '0435', '0438', '0461', '0463', '0467', '0469', '0472', '0475', '0478', '0481', '0483', '0487', '0489', '0492', '0495', '0498', '0513', '0514', '0517', '0519', '0531', '0534', '0537', '0539', '0541', '0543', '0547', '0549', '0571', '0573', '0574', '0579', '0591', '0593', '0594', '0597', '0613', '0614', '0617', '0619', '0631', '0634', '0637', '0639', '0641', '0643', '0647', '0649', '0671', '0673', '0674', '0679', '0691', '0693', '0694', '0697', '0712', '0715', '0718', '0721', '0723', '0724', '0729', '0732', '0735', '0738

,guess,guess_result,guess_actual_entropy,candidate_count,candidate_entropy,best_guess,best_guess_entropy
0,,,NaN,5040,12.299208,,NaN
1,8256,"(0, 1)",1.807355,1440,10.491853,0123,2.858865
2,0123,"(0, 2)",1.964376,369,8.527477,1540,3.085136
3,1540,"(1, 2)",3.942515,24,4.584963,1064,2.953243
4,1064,"(0, 4)",3.584963,2,1.000000,4610,1.000000
5,4610,"(2, 2)",1.000000,1,0.000000,6410,0.000000
6,6410,"(4, 0)",0.000000,1,0.000000,0123,0.000000
